In [12]:

# 3rd 과제, 영화이름으로 영화에 대한 정보를 답장하는 체인
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=50,
    return_messages=True,
)

examples = [
    {
        "movie": "Top Gun",
        "answer": """
        Here are three emojis:
        🛩️👨‍✈️🔥
        """,
    },
    {
        "movie": "The Godfather",
        "answer": """
        Here are three emojis:
        👨‍👨‍👦🔫🍝 
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "You should take the title of a movie and reply with three emojis that represent the {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give three emojis that represent the movie."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "You should take the title of a movie and reply with three emojis that represent the{movie}?"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input" : movie },
        {"output" : result.content},
    )
    print(result)

invoke_chain("Titanic")

invoke_chain("Frozen")

Here are three emojis:
🚢💔🎶content='Here are three emojis:\n🚢💔🎶'
Here are three emojis:
❄️👸⛄content='Here are three emojis:\n❄️👸⛄'


In [16]:
invoke_chain("Please tell me what is the movies I asked first?")

The movie you asked about first is "Titanic."content='The movie you asked about first is "Titanic."'
